### Simple RNN

In the previous notebook we learnt how to create a simple `RNN` that was able to translate text from english to french. This time around we are going to exand that even more by creating an Embedding ``GRU`` that will be able to perform the task, even better. Note that the implementation is the same even when you are using the `LSTM`.

**Note**: The rest of the notebook will remain the same, when there's a change i will highlight.

### Imports

In [1]:
from collections import Counter
import numpy as np
import helper, os, time

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.__version__

'2.5.0'

### Mounting the Google Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Paths to the files

In [3]:
base_path = '/content/drive/MyDrive/NLP Data/seq2seq/fr-en-small'
en_path = 'small_vocab_en.txt'
fr_path = 'small_vocab_fr.txt'

### Loading the data.

We have two files that are located at this path `'/content/drive/MyDrive/NLP Data/seq2seq/fr-en-small'` and thes files are:

```
small_vocab_fr.txt
small_vocab_en.txt
```

The following line help us to load the data.

In [4]:
eng_sents = open(os.path.join(base_path, en_path), encoding='utf8').read().split('\n')
fre_sents = open(os.path.join(base_path, fr_path), encoding='utf8').read().split('\n')

print("Data Loaded")

Data Loaded


In [5]:
eng_sents[1]

'the united states is usually chilly during july , and it is usually freezing in november .'

By looking at the data we can see that the data is already preprocessed, which means we are not going to do that step here.

### Next, Bulding the Vocabulary.

Vocabulary in my definition is just unique words in the curpus. Let's look at the vocabulary size of french and english. But first we need to tokenize each sentence, Inorder for us to do that I'm going to use the `spacy` library which is my favourite when it comes to tokenization of languages.

In [6]:
import spacy
spacy.cli.download('fr_core_news_sm')

spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')


✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [7]:
def tokenize_fr(sent):
  return [tok.text for tok in spacy_fr.tokenizer(sent)]
  
def tokenize_en(sent):
  return [tok.text for tok in spacy_en.tokenizer(sent)]

In [8]:
en_counter = Counter()
fr_counter = Counter()

for sent in eng_sents:
  en_counter.update(tokenize_en(sent.lower()))
for sent in fre_sents:
  fr_counter.update(tokenize_fr(sent.lower()))

In [9]:
en_vocab_size = len(en_counter)
fr_vocab_size = len(fr_counter)

fr_vocab_size, en_vocab_size

(340, 201)

Here we have `340` unique words for french in this dataset and `201` unique words for english.

### Preprocessing.

We will convert our text data into sequence of integers so basically we are going to perform the following:

1. Tokenize the words into ids
2. Pad the tokens so that they will have same length.

For this task we are going to use the keras `Tokenizer` class to perform the task, We have been using this for sentiment analyisis so the procedure is the same.

We are going to have two tokenizers for each language.


In [10]:
en_tokenizer = Tokenizer(num_words=en_vocab_size, oov_token="<oov>")
en_tokenizer.fit_on_texts(eng_sents)

fr_tokenizer = Tokenizer(num_words=fr_vocab_size, oov_token="<oov>")
fr_tokenizer.fit_on_texts(fre_sents)

In [11]:
en_word_indices = en_tokenizer.word_index
en_word_indices_reversed = dict([
    (v, k) for (k, v) in en_word_indices.items()
])

fr_word_indices = fr_tokenizer.word_index
fr_word_indices_reversed = dict([
    (v, k) for (k, v) in fr_word_indices.items()
])

### Helper functions
We will create some helper function that converts sequences to text and text to sequences for each language. These function will be used for inference later on.

**We have set the out of vocabulary `oov_token|| <"oov">`token to `1`  which means the word that does not exist in the vocabulary it's integer representation is 1**

In [12]:
def en_seq_to_text(sequences):
  return " ".join(en_word_indices_reversed[i] for i in sequences )

def en_seq_to_text(sequences):
  return " ".join(fr_word_indices_reversed[i] for i in sequences )

def en_text_to_seq(sent):
  words = tokenize_en(sent.lower())
  sequences = []
  for word in words:
    try:
      sequences.append(en_word_indices[word])
    except:
      sequences.append(1)
  return sequences

def fr_text_to_seq(sent):
  words = tokenize_fr(sent.lower())
  sequences = []
  for word in words:
    try:
      sequences.append(fr_word_indices[word])
    except:
      sequences.append(1)
  return sequences

### Converting text to sequences

In [13]:
en_sequences = en_tokenizer.texts_to_sequences(eng_sents)
fr_sequences = fr_tokenizer.texts_to_sequences(fre_sents)

In [14]:
fr_sequences[0:4]

[[36, 35, 2, 9, 68, 38, 12, 25, 7, 4, 2, 113, 3, 51],
 [5, 33, 32, 2, 13, 20, 3, 50, 7, 4, 96, 70, 3, 52],
 [102, 2, 13, 68, 3, 46, 7, 4, 2, 13, 22, 3, 42],
 [5, 33, 32, 2, 9, 270, 3, 42, 7, 4, 104, 20, 3, 49]]

### Padding Sequences.

In our case we are going to assume that the longest sentence has `100` words for both `fr` and `en` languages.

In [15]:
max_words = 100
en_tokens_padded = pad_sequences(
    en_sequences, 
    maxlen=max_words, 
    padding="post", 
    truncating="post"
)
fr_tokens_padded = pad_sequences(
    fr_sequences, 
    maxlen=max_words, 
    padding="post", 
    truncating="post"
)

In [16]:
en_tokens_padded[:2]

array([[18, 24,  2,  9, 68,  5, 40,  8,  4,  2, 56,  3, 45,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 6, 21, 22,  2, 10, 63,  5, 44,  8,  4,  2, 10, 52,  3, 46,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

### Logits to text.

We are going to create 1 more helper function that will help us to take logits or the predictions probabilities and then we convert them to human understandable format.

In [17]:

def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id
                    in tokenizer.word_index.items()}
  index_to_words[0] = '<pad>'
  """
  For every prediction we are going to ignore the pad token
  """
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)]).replace("<pad>", "")


### Embedding GRU

Unlike from the previous Notebook we never had an Embedding layer. This time we will be having the embedding layer as our first layer in the `Sequential` model.

This is a simple RNN that will learn to translate english sentences to french.

![img](https://github.com/LeanManager/Machine_Translation/raw/e6567f10a6e380eea453fa392de94f26973c8b16/images/embedding.png)

### What is the purpose of the embbeding layer?
The embedding layer is a layer that groups words with the similar meaning close to each other in a vector space. This helps the model to learn better and make better predictions, because if the words with similar meaning have are closer to each other interms of euclidian distance then the model will be able to ofcause notice that and learn better and faster.

We are going to use the Dequential API this time around.

In [20]:

model = keras.Sequential([
    keras.layers.Embedding(
        en_vocab_size,
        128, 
        input_length=max_words
    ),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(256, return_sequences=True),
    keras.layers.GRU(512, return_sequences=True),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dense(fr_vocab_size, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          25728     
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 128)          99072     
_________________________________________________________________
gru_2 (GRU)                  (None, 100, 256)          296448    
_________________________________________________________________
gru_3 (GRU)                  (None, 100, 512)          1182720   
_________________________________________________________________
dense_1 (Dense)              (None, 100, 1024)         525312    
_________________________________________________________________
dense_2 (Dense)              (None, 100, 340)          348500    
Total params: 2,477,780
Trainable params: 2,477,780
Non-trainable params: 0
____________________________________________

In [21]:
tmp_x = en_tokens_padded.reshape(
   -1, 100, 1
)
tmp_x.shape

(137861, 100, 1)

In [23]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(tmp_x, 
          fr_tokens_padded, 
          batch_size=1024, 
          epochs=10,
          validation_split=0.2
)


Epoch 1/10
108/108 [==============================] - 120s 1s/step - loss: 0.5654 - accuracy: 0.8847 - val_loss: 0.4026 - val_accuracy: 0.9037
Epoch 2/10
108/108 [==============================] - 114s 1s/step - loss: 0.2972 - accuracy: 0.9215 - val_loss: 0.2309 - val_accuracy: 0.9368
Epoch 3/10
108/108 [==============================] - 115s 1s/step - loss: 0.1906 - accuracy: 0.9464 - val_loss: 0.1584 - val_accuracy: 0.9542
Epoch 4/10
108/108 [==============================] - 115s 1s/step - loss: 0.1360 - accuracy: 0.9603 - val_loss: 0.1181 - val_accuracy: 0.9650
Epoch 5/10
108/108 [==============================] - 115s 1s/step - loss: 0.1047 - accuracy: 0.9679 - val_loss: 0.0995 - val_accuracy: 0.9690
Epoch 6/10
108/108 [==============================] - 115s 1s/step - loss: 0.0873 - accuracy: 0.9723 - val_loss: 0.0810 - val_accuracy: 0.9744
Epoch 7/10
108/108 [==============================] - 115s 1s/step - loss: 0.0754 - accuracy: 0.9755 - val_loss: 0.0704 - val_accuracy: 0.9771

The rest of the notebook will remain the same, I believe we can load the 
word pretrained word embeddings in our embedding layer as well, but that will be later on.

### Making some predictions.
Our model is targeting to predict french words, during the predict function we are going to do the following:

1. Get the sequence of the english sentence 
2. Pad the english sequences and pass them to the model'
3. Reshape the logits output to the shape of `(max_len, trg_vocabsize(french)`
4. Call the `logits_to_text` function and pass the tokenizer as the `fr_tokenizer`.
5. Get the predictions

In [24]:
def predict(sent):
  sequences = en_text_to_seq(sent)
  padded_tokens = pad_sequences([sequences], maxlen=max_words, padding="post", truncating="post")
  logits = model(padded_tokens)
  logits = tf.reshape(logits, (100, -1))
  return logits_to_text(logits, fr_tokenizer)
predict("your least liked fruit is the grape.")

'votre fruit aimé des fruits la raisin raisin                                                                                            '

### Making more predictions.


In [25]:
from prettytable import PrettyTable
def tabulate_translations(column_names, data, title, max_characters=25):
  table = PrettyTable(column_names)
  table.title= title
  table.align[column_names[0]] = 'l'
  table.align[column_names[1]] = 'l'
  table.align[column_names[2]] = 'l'
  table._max_width = {column_names[0] :max_characters, column_names[1] :max_characters, column_names[2]:max_characters}
  for row in data:
    table.add_row(row)
  print(table)
columns_names = [
    "English (real src sentence)", "French (the actual text)", "Translated (translated version)"
]
title = "ENGLISH TO FRENCH TRANSLATOR"

In [26]:
max_characters= 25
total_translations= 10
for i, (eng, fre) in enumerate(zip(eng_sents[:total_translations], fre_sents)):
    rows_data = [[eng, fre, predict(eng)]]
    if i + 1 != total_translations:
      rows_data.append(["-" * max_characters, "-" * max_characters, "-" * max_characters ])
    tabulate_translations(columns_names, rows_data, title, max_characters)

+-------------------------------------------------------------------------------------------+
|                                ENGLISH TO FRENCH TRANSLATOR                               |
+-----------------------------+---------------------------+---------------------------------+
| English (real src sentence) | French (the actual text)  | Translated (translated version) |
+-----------------------------+---------------------------+---------------------------------+
| new jersey is sometimes     | new jersey est parfois    | new jersey est parfois calme en |
| quiet during autumn , and   | calme pendant l' automne  | l' de et automne et il est      |
| it is snowy in april .      | , et il est neigeux en    | neigeux  en                     |
|                             | avril .                   |                                 |
| -------------------------   | ------------------------- | -------------------------       |
+-----------------------------+---------------------------+-

### Conclusion.

In this notebook we have leant how to create a simple GRU Sequential model that translate text from `eng` to french and we were able to get reasonable accuracy and better and reasonable translation at the end.

### Next
In the next Notebook we will have a look on how we can create a `BidirectionalLSTM` that will perform even better after training for few epochs than this current model.
